## Get API key

In [2]:
import os
import getpass

# Set the environment variable
os.environ["COHERE_API_KEY"] = getpass.getpass("Enter your Cohere API key: ")

In [ ]:
# nt9wg84wNjGfVEwC9ysQuAVdEJlT9bDYjtMeMA8g

In [3]:
cohere_api_key = os.environ["COHERE_API_KEY"]
print(cohere_api_key)

nt9wg84wNjGfVEwC9ysQuAVdEJlT9bDYjtMeMA8g


## load Cohere model 

In [ ]:
from langchain_cohere.llms import Cohere

# model = Cohere(temperature=0.1)
model = Cohere(cohere_api_key=cohere_api_key, model="command", temperature=0.1)

In [4]:
from langchain_cohere import ChatCohere

chat = ChatCohere(cohere_api_key=cohere_api_key, model="command-r-plus", temperature=0.1)

## Prompt Template

In [10]:
from langchain_core.prompts import ChatPromptTemplate

message_prompt = ChatPromptTemplate([
    ("system", "تو یک معلم ریاضی هستی. سوالی که ازت پرسیده می‌شود را مثل یک معلم توضیح بده"),
    ("human", "{first_number} به علاوه‌ی {second_number} چند می‌شود؟")
])

response = chat.invoke(message_prompt.invoke({"first_number": "دویست و پنجاه", "second_number": "صد و پنجاه"}))

print(response.content)

ببینیم اگر دویست و پنجاه و صد و پنجاه را با هم جمع کنیم، چه عددی به دست می‌آید. می‌توانیم این مسئله را به صورت یک مسئله جمع نوشتاری در نظر بگیریم.

دویست و پنجاه به علاوه‌ی صد و پنجاه

اول می‌توانیم قسمت‌های یکسان را به هم جمع کنیم:

دویست به علاوه‌ی صد = سیصد

و پنجاه به علاوه‌ی پنجاه = صد

حالا می‌توانیم نتیجه را به صورت یک عدد بنویسیم:

سیصد به علاوه‌ی صد = چهارصد

پس پاسخ این است که دویست و پنجاه به علاوه‌ی صد و پنجاه برابر است با چهارصد.


In [9]:
import time

for chunk in chat.stream(message_prompt.invoke({"first_number": "دویست و پنجاه", "second_number": "صد و پنجاه"})):
    print(chunk.content, end="", flush=True)
    time.sleep(0.08)


ببینیم اگر دویست و پنجاه و صد و پنجاه را با هم جمع کنیم، چه عددی به دست می‌آید. برای این کار، ابتدا ارقام یکان را با هم جمع می‌کنیم. در اینجا، هر دو عدد پنج دارند، که جمع آنها ده می‌شود. حالا یک صفر را باید نگه داریم و ده را به دهگان منتقل کنیم. پس یکان عدد ما صفر می‌شود.

حالا نوبت به جمع ارقام دهگان می‌رسد. در دویست و پنجاه، پنج را با صد و پنجاه جمع می‌کنیم که می‌شود صد و پنج. به این عدد، ده را که قبلا به دست آورده بودیم، اضافه می‌کنیم که می‌شود صد و پانزده. پس دهگان عدد ما پانزده می‌شود.

در نهایت، چون در بخش صدگان هر دو عدد دو دارند، پس صدگان عدد ما هم دو می‌شود.

بنابراین، دویست و پنجاه به علاوه‌ی صد و پنجاه برابر است با چهارصد و پانزده.

In [16]:
prompt_template = ChatPromptTemplate.from_template(
    "Generate a very short conversation between two people about {topic}"
)

response = chat.invoke(prompt_template.format(topic = "weather"))
print(response.content)

Person 1: "Looks like it's going to rain today."

Person 2: "Yeah, I heard there's a storm coming our way. Hopefully, it won't be too bad."

Person 1: "Let's hope so. I'm not a fan of heavy rain."


## Chain

In [9]:
chain = message_prompt | chat

In [12]:
chain_response = chain.invoke({"first_number": "دویست و پنجاه", "second_number": "صد و پنجاه"})

print(chain_response.content)

ببینیم اگر دویست و پنجاه و صد و پنجاه را با هم جمع کنیم، چه عددی به دست می‌آید. می‌توانیم این مسئله را به صورت یک مسئلهٔ جمع نوشتاری حل کنیم.

دویست و پنجاه به علاوهٔ صد و پنجاه

اول عددهای یکان را با هم جمع می‌کنیم:

50 + 50 = 100

حالا عددهای دهگان را با هم جمع می‌کنیم:

200 + 100 = 300

پس پاسخ این است که دویست و پنجاه به علاوهٔ صد و پنجاه برابر است با سیصد.


## Parallel chain

In [23]:
import markdown
from IPython.display import display, HTML

def print_md_v2(content):
    content = markdown.markdown(content)
    styled_content = f"""
    <div dir="rtl" style="font-family: Tahoma, Arial, sans-serif; font-size: 16px; color: #300; background-color: #f9f9f9; padding: 10px; border-radius: 8px;">
        {content}
    </div>
    """
    display(HTML(styled_content))

In [25]:
message_prompt_poem = ChatPromptTemplate.from_messages(
    [('system', "تو یک شاعر ایرانی هستی"),
     ('human', "یک شعر دوبیتی درباره {topic} بگو")
    ]
)

chain_poem = message_prompt_poem | chat

%time
chain_response_poem = chain_poem.invoke({"topic": "پاییز"})
print_md_v2(chain_response_poem.content)

CPU times: user 2 μs, sys: 0 ns, total: 2 μs
Wall time: 4.05 μs


In [24]:
message_prompt_story = ChatPromptTemplate.from_messages(
    [('system', "تو یک نویسنده ایرانی هستی"),
     ('human', "یک داستان کوتاه ۵ خطی درباره {topic} بگو")
    ]
)

chain_story = message_prompt_story | chat

%time
chain_response_poem = chain_story.invoke({"topic": "پاییز"})
print_md_v2(chain_response_poem.content)

CPU times: user 1 μs, sys: 0 ns, total: 1 μs
Wall time: 2.38 μs


In [34]:
from langchain_core.runnables import RunnableParallel

combined = RunnableParallel(story=chain_story, poem=chain_poem)

response = combined.invoke({"topic": "پاییز"})
for key in response.keys():
    print(key)
    print_md_v2(combined.invoke({"topic": "پاییز"})[key].content)

story


poem


## Runnable lambda

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

message_prompt = ChatPromptTemplate.from_messages([
    ("system", "تو یک معلم ریاضی هستی. سوالی که ازت پرسیده می‌شود را مثل یک معلم توضیح بده"),
    ("human", "عدد {first_number} به علاوه‌ی {second_number} چند می‌شود؟")
])
model_chain = message_prompt | chat | StrOutputParser()

In [30]:
first_number, second_number = "۱۲", "شصت و دو"

chain_response = model_chain.invoke({"first_number": first_number, "second_number": second_number})

print(chain_response)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "first_number": "۱۲",
  "second_number": "شصت و دو"
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "first_number": "۱۲",
  "second_number": "شصت و دو"
}
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatCohere] Entering LLM run with input:
{
  "prompts": [
    "System: تو یک معلم ریاضی هستی. سوالی که ازت پرسیده می‌شود را مثل یک معلم توضیح بده\nHuman: عدد ۱۲ به علاوه‌ی شصت و دو چند می‌شود؟"
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatCohere] [8.06s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "خب بچه‌ها، برای حل این مسئله باید دو عدد داده‌شده را با هم جمع کنیم. توجه داشته باشید که عدد ۱۲ یک عدد دو رقمی است و عدد ۶۲ یک عدد سه رقمی. وقتی با اعداد دو رقمی و سه رقمی سر و کار داریم، باید از روش جمع ستونی استفاده کنیم

In [65]:
import re

def count_numbers(input):

    ## Find all numbers in the text input
    pattern = r'[0-9۰-۹]+'
    numbers = re.findall(pattern, input)

    ## Count the numbers
    return str(len(numbers))

In [67]:
from langchain_core.runnables import RunnableLambda

count_numbers_runnable = RunnableLambda(count_numbers)

count_nums_model = model_chain | count_numbers_runnable

chain_response = count_nums_model.invoke({"first_number": first_number, "second_number": second_number})

print(chain_response)


16


## Return without change - RunnablePassThrough

In [62]:
from langchain_core.runnables import RunnablePassthrough

runnable = RunnableParallel(
    original = RunnablePassthrough(),
    length = lambda x: len(x['input'])
)

runnable.invoke({'input': 'Quera1111'})

{'original': {'input': 'Quera1111'}, 'length': 9}

### RunnablePassThrough - assign

In [63]:
runnable = RunnableParallel(
    original = RunnablePassthrough().assign(lower=lambda x: x['input'].lower()),
    length = lambda x: len(x['input'])
)

runnable.invoke({'input': 'Quera1111'})

{'original': {'input': 'Quera1111', 'lower': 'quera1111'}, 'length': 9}

In [68]:
from langchain_core.runnables import RunnableParallel

final_output = RunnableParallel({
    "model_answer": RunnablePassthrough(),  
    "count_numbers_answer": count_numbers_runnable 
})

combined_chain = model_chain | final_output

response = combined_chain.invoke({"first_number": first_number, "second_number": second_number})
response

{'model_answer': 'ببینیم اگر این دو عدد را با هم جمع کنیم به چه عددی می\u200cرسیم.\n\nعدد ۱۲ را می\u200cنویسیم و کنار آن عدد ۶۲ را می\u200cنویسیم:\n\n۱۲ + ۶۲ =\n\nحالا یکان\u200cهای این دو عدد را با هم جمع می\u200cکنیم:\n\n۲ + ۲ = ۴\n\nعدد ۴ را به عنوان یکان عدد جواب می\u200cنویسیم.\n\nحالا دهگان\u200cهای این دو عدد را با هم جمع می\u200cکنیم. دهگان عدد ۱۲، صفر است، پس فقط عدد ۶ را می\u200cنویسیم:\n\n۶\n\nحالا عدد ۶ را با ۴ یکان که قبلا به دست آورده\u200cایم، کنار هم می\u200cنویسیم:\n\n۷۴\n\nپس می\u200cبینیم که ۱۲ به علاوه\u200cی ۶۲ برابر است با ۷۴.',
 'count_numbers_answer': '17'}

## Debug

In [14]:
from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

person_prompt = ChatPromptTemplate.from_template("what is the city {person} is from?")

sub_chain = person_prompt | chat | StrOutputParser()

city_prompt = ChatPromptTemplate.from_template(
    "what country is the city {city} in? respond in {language}"
)

chain = (
    {"city": sub_chain, "language": itemgetter("language")}
    | city_prompt
    | chat
    | StrOutputParser()
)

chain = chain.with_config({"verbose": True})

In [ ]:
response = chain.invoke({"person": "Beethoven", "language": "english"}, )
print(response)

Germany.


In [22]:
chain.get_graph().print_ascii()

           +------------------------------+      
           | Parallel<city,language>Input |      
           +------------------------------+      
                  ***             ***            
                **                   ***         
              **                        **       
+--------------------+                    **     
| ChatPromptTemplate |                     *     
+--------------------+                     *     
           *                               *     
           *                               *     
           *                               *     
    +------------+                         *     
    | ChatCohere |                         *     
    +------------+                         *     
           *                               *     
           *                               *     
           *                               *     
  +-----------------+                 +--------+ 
  | StrOutputParser |                 | Lambda | 


## Callback

✅ What Is a Callback (General Definition)?

A callback is a function (or method) that you pass to another piece of code, and it gets called automatically when something happens.

👉 Think of it as saying:

"Hey, when you’re done doing X, call me and I’ll do Y."

Callbacks are common in:

- Event-driven programming (e.g., GUIs, clicking buttons)
- Asynchronous code (e.g., when a request finishes)
- Logging/debugging (e.g., when a process starts or ends)
- Custom behavior hooks (e.g., adding your own logic to an existing process)

In [28]:
from langchain.callbacks.tracers import ConsoleCallbackHandler

response = chain.invoke({"person": "Beethoven", "language": "english"})#, config={"callbacks": [ConsoleCallbackHandler()]})
print(response)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "person": "Beethoven",
  "language": "english"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<city,language>] Entering Chain run with input:
{
  "person": "Beethoven",
  "language": "english"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<city,language> > chain:RunnableSequence] Entering Chain run with input:
{
  "person": "Beethoven",
  "language": "english"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<city,language> > chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "person": "Beethoven",
  "language": "english"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<city,language> > chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > chain:RunnableParallel<city,language> > chain:RunnableSequence > llm:ChatCohere] Entering LLM ru

## Set debug

In [26]:
from langchain.globals import set_debug
set_debug(True)